<a href="https://colab.research.google.com/github/ArielFerreiro/ar-stamp-checker/blob/main/attempt.01/02_1_StampsCrop_w_BBoxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using the stamp detector model we use it to create a new cropper based on the automatic annotations done by that model

In [ ]:
%load_ext autoreload
%autoreload 2

#### Load the drive to grab the models and images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Notebook Imports

In [ ]:
import pandas as pd
import numpy as np
import pickle
import cv2
from os import path
from matplotlib import pyplot as plt
from keras.models import load_model

#### Global Params

In [ ]:
images_path = 'drive/My Drive/Colab Notebooks/stamps/original/'
results_path = 'drive/My Drive/Colab Notebooks/stamps/results/'
models_path = 'drive/My Drive/Colab Notebooks/stamps/models/'

#### Notebook Functions

In [ ]:
def cropImage(path_in, item, model):
    """
    Opens an image from the original dataset, crops it using the model, 
    resize the picture and returns it as a numpy array together with its class
    """
    dim = (299, 299)
    image_in = path_in+item.label+"/"+item.file_name

    classe = 0 #genuine
    if item.label[0:3]=='cft':
      classe = 1 #fake
    
    if path.exists(image_in):  

        # open image
        img = cv2.imread(image_in)
        map = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        #save original image data
        original_image = map
        original_height = map.shape[0]
        original_width = map.shape[1]

        # Prepare the image to enter the model
        map = cv2.resize(map, dsize=(224,224))
        map = map / map.max() #255
        map = np.expand_dims(map, axis=0)

        # Predict BBOX (Crop)
        pred_bbox = model.predict(map)
        bbox = pred_bbox[0]*193.0
        x, y, w, h = bbox

        # Adjust box to original image size
        new_x = original_width * x / 224
        new_y =  original_height * y / 224
        new_w = original_width * w / 244
        new_h = original_height * h / 224

        # crop image
        crop_img = original_image[int(new_y):int(new_y)+int(new_h), int(new_x):int(new_x)+int(new_w)]
        
        # resize image
        resized = cv2.resize(crop_img, dim, interpolation = cv2.INTER_AREA)
        
        return resized, classe
    else:
        return None, None

#### Import *Model*

In [ ]:
bbox_model_name = 'vgg19_bbox_detector.h5'

# Bounding Box Detector Model -> Detects the position of the stamp in a full pic
bbox_detector =  load_model(models_path + bbox_model_name)
bbox_detector.load_weights(models_path + 'weights_' + bbox_model_name)

#### Start

In [ ]:
with open(results_path+"labels.pkl","rb") as f:
    (df, classes_idx, cant_classes, classes) = pickle.load(f)

In [ ]:
stamps = []
y = []
for index, row in df.iterrows():
    stamp, classe = cropImage(images_path, row, bbox_detector)
    if not stamp is None:
      stamps.append(stamp)
      y.append(classe)

In [ ]:
len(stamps)

In [ ]:
stamps_array = np.array(stamps)
y_array = np.array(y)
stamps_array.shape

#### Checkpoint

In [ ]:
plt.imshow(stamps_array[0])

In [ ]:
with open(results_path + "stamps_bb_299.pkl","wb") as f:
    pickle.dump((stamps_array, y_array), f)

In [ ]:
#### DONE v1 02/11/2020
#### Auth: Ariel Ferreiro Diaz